
# Análisis descriptivo, tratamiento de valores faltantes y variables categóricas


## Importar datos

Cargamos las librerías necesarias.
- `pandas` y `numpy` para manipulación de datos,
- `matplotlib` para visualizaciones,
- `sklearn` para imputación y One-Hot Encoding.


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer



## 1) Carga del CSV (`datos.csv` con separador `;`)


In [ ]:
df = pd.read_csv("spotify-2023.csv")
df



## 2) Inspección inicial

Preguntas guía:
- ¿Cuántas observaciones y variables hay?
- ¿Qué tipos de datos tenemos (numéricos, cadenas/objetos, fechas)?
- ¿Hay columnas con un único valor o muchas categorías?


In [ ]:

print("Dimensiones:", df.shape)
print("\nTipos de datos:")
print(df.dtypes)

print("\nPrimeras filas:")
display(df.head())

print("\nValores únicos aproximados por columna (hasta 20 ejemplos):")
for col in df.columns:
    uniq = df[col].unique()
    print(f"- {col}: {len(uniq)} únicos. Valores:", uniq[:20])



## 3) Análisis descriptivo

- Estadísticos básicos para variables **numéricas** con `describe()`.
- Resumen de frecuencias para variables **categóricas** (tipo `object`/`category`).

> Nota: Si tienes valores numéricos almacenados como texto con comas/puntos, considera convertirlos previamente.


In [ ]:

print("Estadísticos descriptivos (numéricos):")
display(df.select_dtypes(include=[np.number]).describe().T)

print("\nFrecuencias de las 10 categorías más comunes por columna categórica:")
cat_cols_raw = df.select_dtypes(include=['object','category']).columns.tolist()
for col in cat_cols_raw:
    print(f"\nColumna: {col}")
    display(df[col].value_counts(dropna=False).head(10))



### Visualizaciones rápidas

Reglas:
- Usamos `matplotlib`.
- **Una gráfica por celda**.
- **Sin estilos ni colores específicos**.

1. Histograma de la **primera columna numérica**.


In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
plt.figure()
df[num_cols[0]].plot(kind='hist', bins=30, title=f'Histograma: {num_cols[0]}')
plt.xlabel(num_cols[0])
plt.ylabel('Frecuencia')
plt.show()




2. Boxplot de la **segunda columna numérica** (si existe).


In [ ]:

plt.figure()
df[[num_cols[1]]].plot(kind='box', title=f'Boxplot: {num_cols[1]}')
plt.ylabel(num_cols[1])
plt.show()



## 4) Detección de valores faltantes

Aquí calculamos el conteo de `NaN` por columna y el porcentaje de faltantes.


In [ ]:

na_counts = df.isna().sum().sort_values(ascending=False)
na_pct = (df.isna().mean()*100).sort_values(ascending=False)

print("Top columnas con más faltantes:")
display(pd.DataFrame({'faltantes': na_counts, 'porcentaje': na_pct}).head(20))



## 5) Imputación de valores faltantes

Estrategia simple (puedes ajustarla):
- Para **numéricos**: imputar con **mediana** (más robusta a outliers que la media).
- Para **categóricos**: imputar con **moda** (valor más frecuente).


In [ ]:

df_impute = df.copy()

num_cols = df_impute.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_impute.select_dtypes(include=['object','category']).columns.tolist()

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

if num_cols:
    df_impute[num_cols] = num_imputer.fit_transform(df_impute[num_cols])
if cat_cols:
    df_impute[cat_cols] = cat_imputer.fit_transform(df_impute[cat_cols])

print("Comprobación de faltantes tras imputación:")
display(df_impute.isna().sum().sort_values(ascending=False).head(10))



## 6) Variables dummies

Dos caminos:
1. **`pandas.get_dummies`** (rápido) con `drop_first=True` para evitar colinealidad perfecta.  
2. **`sklearn.OneHotEncoder`** dentro de un `Pipeline` + `ColumnTransformer` (útil para modelado).

Primero, normalizamos tipos categóricos a `category`.


In [ ]:

df_feat = df_impute.copy()

# Asegurar tipo categórico
for c in cat_cols:
    df_feat[c] = df_feat[c].astype('category')

# 6.1 Dummies con pandas
df_dummies_pd = pd.get_dummies(df_feat, columns=cat_cols, drop_first=True)
print("Columnas añadidas (muestra) con get_dummies:")
display([c for c in df_dummies_pd.columns if any(c.startswith(col+'_') for col in cat_cols)][:20])
display(df_dummies_pd.head())



## 7) Exportación del dataset limpio

Guardamos dos versiones:
- `datos_limpios.csv` a partir de **get_dummies (pandas)**.


In [ ]:

df_dummies_pd.to_csv("datos_limpios.csv", index=False)
